In [13]:
# Import all we need

import keras
import tensorflow as tf
from keras import layers
#from keras.datasets import mnist, fashion_mnist
import numpy as np
from ipywidgets import interact
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
from PIL import Image
from tensorflow.python.layers.convolutional import conv2d

print ('import completed')

import completed


In [14]:
batch_size = 128
img_wid, img_hei = 256, 256
global a

def make_paths(dir_in):
    path_input = os.path.join(dir_in, 'input')
    path_output = os.path.join(dir_in, 'output')
    return path_input, path_output

def create_dataset(path):
    imgs_in, imgs_out = make_paths(path)
    img_in_paths = sorted(glob.glob(os.path.join(imgs_in, '*.bmp')))
    img_out_paths = sorted(glob.glob(os.path.join(imgs_out, '*.bmp')))

    dataset = tf.data.Dataset.from_tensor_slices((img_in_paths, img_out_paths))
    dataset = dataset.map(lambda a, b: load_pair(a, b), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).shuffle(100).prefetch(tf.data.AUTOTUNE)
    return dataset

def load_pair(input_path, output_path):
    input_image = load_image(input_path)
    output_image = load_image(output_path)
    return input_image, output_image

def load_image(path):

    img = tf.io.read_file(path)
    global a
    a = path
    img = tf.image.decode_bmp(img, channels=3)
    # resizing not necessary
    img = tf.cast(img, tf.float32) / 255.0
    return img


train_dir = '/home/maciejka/Desktop/school/S8/labwork-project/db/dataset_preprocessed/train'
val_dir = '/home/maciejka/Desktop/school/S8/labwork-project/db/dataset_preprocessed/val'
test_dir = '/home/maciejka/Desktop/school/S8/labwork-project/db/dataset_preprocessed/test'

train_ds = create_dataset(train_dir)
val_ds = create_dataset(val_dir)
test_ds = create_dataset(test_dir)
i = 0
#for x in train_ds.take(2):
#    fig, ax = plt.subplots(1, 2)
#    ax[0].imshow(x[0])
#    ax[1].imshow(x[1])
#    ax[1].axis('off')
#    ax[0].axis('off')



In [18]:
# MOdel

original_dim = img_hei * img_wid
latent_dim = 256

lr_sz = [16, 32, 64]

def encoder():
    input_img = keras.Input(shape=(img_hei, img_wid, 3))
    # Encoder
    lrs = layers.Conv2D(lr_sz[0], (3, 3), strides = 2, padding='same')(input_img)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    lrs = layers.Conv2D(lr_sz[1], (3, 3), strides = 2, padding='same')(lrs)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    lrs = layers.Conv2D(lr_sz[2], (3, 3), strides = 2, padding='same')(lrs)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    # flattening to latent space
    lrs = layers.Reshape([-1])(lrs)
    mu = layers.Dense(latent_dim, name="mu")(lrs)
    sigma = layers.Dense(latent_dim, name="sigma")(lrs)

    def sampling(args):
        mu, sigma = args
        epsilon = tf.random.normal(shape=(tf.shape(mu)[0], latent_dim))
        return mu + tf.exp(sigma / 2) * epsilon

    z = layers.Lambda(sampling, name="z", output_shape=(latent_dim,))([mu, sigma])

    return keras.Model(input_img, [mu, sigma, z], name="encoder")

def decoder():
    latent_input = layers.Input(shape=(latent_dim,))
    lrs = layers.Dense(16*16*256)(latent_input)
    lrs = layers.Reshape((16, 16, 256))(lrs)

    lrs = layers.Conv2DTranspose(lr_sz[2], (3, 3), strides = 2, padding='same')(lrs)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    lrs = layers.Conv2DTranspose(lr_sz[1], (3, 3), strides = 2, padding='same')(lrs)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    lrs = layers.Conv2DTranspose(lr_sz[0], (3, 3), strides = 2, padding='same')(lrs)
    lrs = layers.BatchNormalization()(lrs)
    lrs = layers.LeakyReLU()(lrs)

    output = layers.Conv2DTranspose(3, (3, 3), strides = 2, padding='same', activation='sigmoid')(lrs)

    return keras.Model(latent_input, output, name="decoder")

# build model

encoder = encoder()
encoder.summary()
decoder = decoder()

get_in_there = layers.Input(shape=(img_wid, img_hei, 3))
mu, sigma, z = encoder(get_in_there)
get_out_from_here = decoder(z)

vae = keras.Model(get_in_there, get_out_from_here, name="vae")


optimizer = keras.optimizers.Adam(
    learning_rate=0.002
)
@tf.function
def loss(y_true, y_pred):
    mse_loss = keras.losses.MeanSquaredError()

    y_true_flattened = tf.reshape(y_true, [-1])
    y_pred_flattened = tf.reshape(y_pred, [-1])

    recon_loss = mse_loss(y_true_flattened, y_pred_flattened)

    recon_loss *= img_wid * img_hei * 3
    kl_loss = -.5 * tf.reduce_sum(1 + sigma - tf.square(mu) - tf.exp(sigma), axis=-1)
    total_loss = tf.reduce_mean(recon_loss + kl_loss)
    return total_loss


vae.compile(optimizer='adam', loss='mse')

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │        448 │ input_layer_9[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         64 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_18      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 64, 64,    │      4,640 │ leaky_re_lu_18[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        128 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_19      │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 32, 32,    │     18,496 │ leaky_re_lu_19[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_20      │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 65536)     │          0 │ leaky_re_lu_20[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mu (Dense)          │ (None, 256)       │ 16,777,472 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sigma (Dense)       │ (None, 256)       │ 16,777,472 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z (Lambda)          │ (None, 256)       │          0 │ mu[0][0],         │
│                     │                   │            │ sigma[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 33,578,976 (128.09 MB)

 Trainable params: 33,578,752 (128.09 MB)

 Non-trainable params: 224 (896.00 B)

In [16]:
vae.summary()

Model: "vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ [(None, 256), (None,   │    33,578,976 │
│                                 │ 256), (None, 256)]     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 256, 256, 3)    │    17,014,243 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,593,219 (193.00 MB)

 Trainable params: 50,592,771 (193.00 MB)

 Non-trainable params: 448 (1.75 KB)

In [17]:
history = vae.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    steps_per_epoch=50,
    validation_steps=50,
    verbose=2
)

Epoch 1/100


/home/maciejka/Desktop/school/S8/labwork-project/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


50/50 - 21s - 423ms/step - loss: 0.1412 - val_loss: 0.1431
Epoch 2/100
50/50 - 16s - 323ms/step - loss: 0.1351 - val_loss: 0.1419
Epoch 3/100
50/50 - 16s - 325ms/step - loss: nan - val_loss: nan
Epoch 4/100
50/50 - 15s - 308ms/step - loss: nan - val_loss: nan
Epoch 5/100
50/50 - 17s - 341ms/step - loss: nan - val_loss: nan
Epoch 6/100
50/50 - 15s - 307ms/step - loss: nan - val_loss: nan
Epoch 7/100
50/50 - 15s - 304ms/step - loss: nan - val_loss: nan
Epoch 8/100
50/50 - 15s - 306ms/step - loss: nan - val_loss: nan
Epoch 9/100
50/50 - 15s - 308ms/step - loss: nan - val_loss: nan
Epoch 10/100
50/50 - 15s - 307ms/step - loss: nan - val_loss: nan
Epoch 11/100
50/50 - 15s - 306ms/step - loss: nan - val_loss: nan
Epoch 12/100
50/50 - 15s - 308ms/step - loss: nan - val_loss: nan
Epoch 13/100
50/50 - 15s - 309ms/step - loss: nan - val_loss: nan
Epoch 14/100
50/50 - 15s - 306ms/step - loss: nan - val_loss: nan
Epoch 15/100
50/50 - 15s - 306ms/step - loss: nan - val_loss: nan
Epoch 16/100
50/50 